<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/openapi_game_engine_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

#Check if runes-client is installed, if not then install it
try:
    import runes_client
except ImportError:
    !pip install runes-client
    import runes_client

try:
    import runes_client
except ImportError:
  !pip install runes-client --upgrade


import runes_client.core as runes
from runes_client import RunesFilePath

placeholder_txt = "Enter the token generated by the Crucible plugin"
RUNES_TOKEN = "065955e9-6c31-40ca-be75-e731caa9ffe2" #@param {type:"string"}
runes_token = RUNES_TOKEN

dn_cli_token = os.getenv('DN_CLI_TOKEN', None)
if dn_cli_token is not None and dn_cli_token != "":
  runes_token = dn_cli_token
else:
  if runes_token is None or runes_token == "" or runes_token == placeholder_txt:
      print("ERROR: The token provided is not valid.")
      exit()

OPENAI_API_KEY = "xxxx" #@param {type:"string"}
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

if os.getenv('OPENAI_API_KEY', None) is None or os.getenv('OPENAI_API_KEY', None) == "":
  print("ERROR: The OpenAI API key is not valid.")
  exit()

# There are two separate function you need to register.
# 1. The imports.  This is where you load any libraries you need to use in your RUNE.  This is important so that the RUNE is aware when dependencies are loaded and its ready to be called from the plugin.
# 2. The method.  This is the primary function of the RUNE.  This is where you write the code that will be executed when the RUNE is triggered.

# Name the function anything you like.  This is the imports function.  This is where you load any libraries you need to use in your RUNE.
async def register_imports():

    # Clone the GitHub repository
    !git clone https://github.com/shiehn/llm_mud.git

    # Change directory to the cloned repo
    %cd llm_mud

    # Install required dependencies
    !pip install -r requirements.txt

    # Import the refactored RPGChat class
    from app import RPGChat

    # Create an instance of the RPGChat class
    global rpg_chat
    rpg_chat = RPGChat()

    #runes.make_imports_global([])


# Now register the imports function with the discovery server.
runes.register_imports(register_imports)

async def runes_func(user_question: str):
    try:

        user_question += " environment_id=def"

        print("HUMAN:", user_question)
        # Send a question to the RPG system and get the response
        response = rpg_chat.ask_question(user_question)

        print("BOT:", response)

        # after executing your custom code you send data back to the plugin like so ..
        #await runes.output().add_file(input_file)
        await runes.output().add_message(response)
        await runes.output().add_log(response)
        await runes.output().send()

        return True
    except Exception as e:
        await runes.output().add_error(f"Error in arbitrary_method: {e}")

        return False


runes.set_token(token=runes_token)
runes.set_name("OpenAPI Chat Engine")
runes.set_description("This is an game chat engine powered by OpenAPI")
runes.register_method(runes_func)

runes.connect_to_server()

KeyboardInterrupt: 